In [1]:
!pip install tika
from tika import parser
import pandas as pd
import numpy as np
import json
from tqdm import tqdm_notebook as tqdm
from multiprocessing import Pool
import hashlib
TIMEOUT=600
import socket
from socket import timeout
socket.setdefaulttimeout(TIMEOUT)
requestOptions={'timeout': TIMEOUT}

In [2]:
def write_json(tika_json, row):
    doi = row['doi']
    clean_doi = str(doi)
    if clean_doi == "nan":
        clean_doi = clean_doi+"_"+str(hashlib.sha224(row['abstract'].encode('utf-8')).hexdigest())
    else:
        clean_doi =  str(doi).replace("/", "_")
    out_file = "COVID-19/ctakes-json/"+clean_doi+".json"
    with open(out_file, 'w') as jw:
        json_obj = json.dumps(tika_json, indent = 4)
        print("Writing JSON to:  ["+out_file+"]")
        jw.write(json_obj)

In [3]:
def parallelize_dataframe(df, func, n_cores=32):
    df_split = np.array_split(df, n_cores)
    pool = Pool(n_cores)
    pool.map(func, df_split)
    pool.close()
    pool.join()

In [4]:
def iter_process_func(df):
    for index, row in tqdm(df.iterrows()):
        try:
            tika_json = parser.from_buffer(row['abstract'], requestOptions=requestOptions, serverEndpoint=tika_url)
            write_json(tika_json, row)
        except Exception as e:
            print("Error parsing: "+str(row['abstract'])+": Msg: "+str(e))

In [5]:
!mkdir COVID-19
!mkdir COVID-19/ctakes-json
!mkdir COVID-19/2020-03-13/

mkdir: cannot create directory ‘COVID-19’: File exists
mkdir: cannot create directory ‘COVID-19/ctakes-json’: File exists
mkdir: cannot create directory ‘COVID-19/2020-03-13/’: File exists


In [6]:
!curl -k -o COVID-19/2020-03-13/metadata.csv https://ai2-semanticscholar-cord-19.s3-us-west-2.amazonaws.com/2020-03-20/metadata.csv

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100 59.8M  100 59.8M    0     0  15.6M      0  0:00:03  0:00:03 --:--:-- 15.6M


In [7]:
covid_all_sources =  "COVID-19/2020-03-13/metadata.csv"

In [8]:
covid_df = pd.read_csv(covid_all_sources)

In [9]:
covid_df

,sha,source_x,title,doi,pmcid,pubmed_id,license,abstract,publish_time,authors,journal,Microsoft Academic Paper ID,WHO #Covidence,has_full_text,full_text_file
0,NaN,Elsevier,Intrauterine virus infections and congenital h...,10.1016/0002-8703(72)90077-4,NaN,4361535.0,els-covid,Abstract The etiologic basis for the vast majo...,1972-12-31,"Overall, James C.",American Heart Journal,NaN,NaN,False,custom_license
1,NaN,Elsevier,Coronaviruses in Balkan nephritis,10.1016/0002-8703(80)90355-5,NaN,6243850.0,els-covid,NaN,1980-03-31,"Georgescu, Leonida; Diosi, Peter; Buţiu, Ioan;...",American Heart Journal,NaN,NaN,False,custom_license
2,NaN,Elsevier,Cigarette smoking and coronary heart disease: ...,10.1016/0002-8703(80)90356-7,NaN,7355701.0,els-covid,NaN,1980-03-31,"Friedman, Gary D",American Heart Journal,NaN,NaN,False,custom_license
3,aecbc613ebdab36753235197ffb4f35734b5ca63,Elsevier,Clinical and immunologic studies in identical ...,10.1016/0002-9343(73)90176-9,NaN,4579077.0,els-covid,"Abstract Middle-aged female identical twins, o...",1973-08-31,"Brunner, Carolyn M.; Horwitz, David A.; Shann,...",The American Journal of Medicine,NaN,NaN,True,custom_license
4,NaN,Elsevier,Epidemiology of community-acquired respiratory...,10.1016/0002-9343(85)90361-4,NaN,4014285.0,els-covid,Abstract Upper respiratory tract infections ar...,1985-06-28,"Garibaldi, Richard A.",The American Journal of Medicine,NaN,NaN,False,custom_license
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
44215,d4f00f66c732c292fcfc28b19f44daa2fa620901,PMC,Epidemiology and clinical profile of pathogens...,10.1371/journal.pone.0188325,PMC5693464,29149199.0,cc-by,This study aimed to identify a broad spectrum ...,2017 Nov 17,"Brini, Ines; Guerrero, Aida; Hannachi, Naila; ...",PLoS One,NaN,NaN,True,comm_use_subset
44216,ec575d33c0d3b34af7644fcfed64af045a75ab63,Elsevier,Functional Analysis of the Transmembrane Domai...,10.1016/j.jmb.2008.12.029,PMC2750892,19121325.0,els-covid,"Abstract To enter cells, enveloped viruses use...",2009-02-13,"Bissonnette, Mei Lin Z.; Donald, Jason E.; DeG...",Journal of Molecular Biology,NaN,NaN,True,custom_license
44217,7f8715a818bfd325bf4413d3c07003d7ce7b6f7e,PMC,Viral Entry Properties Required for Fitness in...,10.1128/mBio.00898-18,PMC6030562,29970463.0,cc-by,Human parainfluenza viruses cause a large burd...,2018 Jul 3,"Iketani, Sho; Shean, Ryan C.; Ferren, Marion; ...",mBio,NaN,NaN,True,comm_use_subset
44218,07e78e218a159c35e9599e3751a99551a271597b,Elsevier,Arenavirus reverse genetics: New approaches fo...,10.1016/j.virol.2011.01.013,PMC3057228,21324503.0,els-covid,"Abstract Several arenaviruses, chiefly Lassa v...",2011-03-15,"Emonet, Sebastien E.; Urata, Shuzo; de la Torr...",Virology,NaN,NaN,True,custom_license


In [10]:
len(covid_df[covid_df.abstract.isnull() != True])

35806

In [11]:
len(covid_df[covid_df['doi'].isnull() != True])

40750

In [12]:
valid_abs_df = covid_df[covid_df.abstract.isnull() != True]

In [13]:
len(valid_abs_df[valid_abs_df.abstract.isnull()])

0

In [14]:
tika_url = 'http://tika.covid19data.space.internal/'

In [15]:
parallelize_dataframe(valid_abs_df, iter_process_func)

/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:2: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  
/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:2: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  


/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:2: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  


/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:2: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  
/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:2: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  
/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:2: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  
/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:2: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  


/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:2: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  


/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:2: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  
/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:2: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  
/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:2: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  
/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:2: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  
/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:2: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  


...

Error parsing: BACKGROUND: Genetic variants in the inhibiting FcγRIIB mediate anti-inflammatory responses and influence IVIG refractoriness (IVIG-R). However, these variants are rare in Asian and Hispanic populations so other genes in the pathway could be potentially involved. IVIG is ineffective in mice lacking SIGN-R1, a related molecule to human DC-SIGN. Further, DC-SIGN is a known receptor for sialylated Fc, the component responsible for the anti-inflammatory action of IVIG. Thus, we hypothesized that DC-SIGN would also be involved in the pathway of IVIG response in Kawasaki Disease (KD) patients. FINDINGS: A case-control approach was performed to examine the differential distribution of five single nucleotide polymorphisms (SNPs) in DC-SIGN promoter with IVIG-R among White (158 vs. 62), Asian (64 vs. 12) and Hispanic (55 vs. 20) KD patients. Distinct differences in allele frequency distributions of several variants in the DC-SIGN promoter were observed in the three ethnic groups. 

2020-03-25 19:44:39,031 [MainThread  ] [WARNI]  Tika server returned status: 504


Error parsing: Summary The magnitude of the 2013–2016 Ebola virus disease (EVD) epidemic enabled an unprecedented number of viral mutations to occur over successive human-to-human transmission events, increasing the probability that adaptation to the human host occurred during the outbreak. We investigated one nonsynonymous mutation, Ebola virus (EBOV) glycoprotein (GP) mutant A82V, for its effect on viral infectivity. This mutation, located at the NPC1-binding site on EBOV GP, occurred early in the 2013–2016 outbreak and rose to high frequency. We found that GP-A82V had heightened ability to infect primate cells, including human dendritic cells. The increased infectivity was restricted to cells that have primate-specific NPC1 sequences at the EBOV interface, suggesting that this mutation was indeed an adaptation to the human host. GP-A82V was associated with increased mortality, consistent with the hypothesis that the heightened intrinsic infectivity of GP-A82V contributed to disease 

2020-03-25 19:54:30,939 [MainThread  ] [WARNI]  Tika server returned status: 504


Error parsing: P001 - Sepsis impairs the capillary response within hypoxic capillaries and decreases erythrocyte oxygen-dependent ATP efflux R. M. Bateman, M. D. Sharpe, J. E. Jagger, C. G. Ellis P002 - Lower serum immunoglobulin G2 level does not predispose to severe flu. J. Solé-Violán, M. López-Rodríguez, E. Herrera-Ramos, J. Ruíz-Hernández, L. Borderías, J. Horcajada, N. González-Quevedo, O. Rajas, M. Briones, F. Rodríguez de Castro, C. Rodríguez Gallego P003 - Brain protective effects of intravenous immunoglobulin through inhibition of complement activation and apoptosis in a rat model of sepsis F. Esen, G. Orhun, P. Ergin Ozcan, E. Senturk, C. Ugur Yilmaz, N. Orhan, N. Arican, M. Kaya, M. Kucukerden, M. Giris, U. Akcan, S. Bilgic Gazioglu, E. Tuzun P004 - Adenosine a1 receptor dysfunction is associated with leukopenia: A possible mechanism for sepsis-induced leukopenia R. Riff, O. Naamani, A. Douvdevani P005 - Analysis of neutrophil by hyper spectral imaging - A preliminary repor